# Homework
[Dogs vs. Cats Redux: Kernels Edition](https://www.kaggle.com/c/dogs-vs-cats-redux-kernels-edition)

In [1]:
%matplotlib inline

import sys
import os

#import modules
sys.path.append('../../deeplearning1/nbs')
from utils import *
from vgg16 import Vgg16

sys.path.append('..')
from env import KAGGLE_USER, KAGGLE_PW, COURSE_DIR

/Users/yangzhang/anaconda/lib/python2.7/site-packages/matplotlib/font_manager.py:273: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')
Using Theano backend.


In [2]:
DOGSCATS_KAGGLE_RAW_DIR = COURSE_DIR+'data/dogscats_kaggle_raw'
DOGSCATS_KAGGLE_DIR = COURSE_DIR+'data/dogscats_kaggle'

# Download kaggle data (do it only once)

In [ ]:
%mkdir -p $DOGSCATS_KAGGLE_RAW_DIR
%cd $DOGSCATS_KAGGLE_RAW_DIR
!pip install kaggle-cli

%%bash -s "$KAGGLE_USER" "$KAGGLE_PW" 
kg config -g -u $1 -p $2 -c dogs-vs-cats-redux-kernels-edition
kg download

unzip test.zip
unzip train.zip

%mkdir -p $DOGSCATS_KAGGLE_DIR

%mv $DOGSCATS_KAGGLE_RAW_DIR'train' $DOGSCATS_KAGGLE_DIR
%mv $DOGSCATS_KAGGLE_RAW_DIR'test' $DOGSCATS_KAGGLE_DIR

# Reorg data (do it only once)

## Create folders

In [ ]:
%cd $DOGSCATS_KAGGLE_DIR
%mkdir valid
%mkdir results
%mkdir -p sample/train
%mkdir -p sample/test/unknown
%mkdir -p sample/valid
%mkdir -p sample/results
%mkdir -p test/unknown

## Move some training files to valid and copy some to sample

In [ ]:
%cd $DOGSCATS_KAGGLE_DIR/train

In [ ]:
g = glob('*.jpg')
shuf = np.random.permutation(g)
for i in range(2000):
    os.rename(shuf[i], DOGSCATS_KAGGLE_DIR+'valid/'+shuf[i])

In [ ]:
from shutil import copyfile

In [ ]:
g = glob('*.jpg')
shuf = np.random.permutation(g)
for i in range(500):
    copyfile(shuf[i], DOGSCATS_KAGGLE_DIR+'sample/train/'+shuf[i])

In [ ]:
%cd $DOGSCATS_KAGGLE_DIR'valid/'

In [ ]:
g = glob('*.jpg')
shuf = np.random.permutation(g)
for i in range(20):
    copyfile(shuf[i], DOGSCATS_KAGGLE_DIR+'sample/valid/'+shuf[i])

In [ ]:
%cd $DOGSCATS_KAGGLE_DIR/test

In [ ]:
g = glob('*.jpg')
shuf = np.random.permutation(g)
for i in range(20):
    copyfile(shuf[i], DOGSCATS_KAGGLE_DIR+'/sample/test/'+shuf[i])

## Rearrange data into labeled folders

In [ ]:
%cd $DOGSCATS_KAGGLE_DIR/train
%mkdir dogs
%mkdir cats
%mv dog.*.jpg dogs/
%mv cat.*.jpg cats/

In [ ]:
%cd $DOGSCATS_KAGGLE_DIR/valid
%mkdir dogs
%mkdir cats
%mv dog.*.jpg dogs/
%mv cat.*.jpg cats/

In [ ]:
%cd $DOGSCATS_KAGGLE_DIR/test
%mv *.jpg unknown/

In [ ]:
%cd $DOGSCATS_KAGGLE_DIR/sample/train
%mkdir dogs
%mkdir cats
%mv dog.*.jpg dogs/
%mv cat.*.jpg cats/

%cd $DOGSCATS_KAGGLE_DIR/sample/valid
%mkdir dogs
%mkdir cats
%mv dog.*.jpg dogs/
%mv cat.*.jpg cats/

%cd $DOGSCATS_KAGGLE_DIR/sample/test
%mv *.jpg unknown/

# Train and Finetune

In [26]:
path = DOGSCATS_KAGGLE_DIR # + '/sample'
test_path = path + '/test/'
results_path = path + '/results/'
train_path = path + '/train/'
valid_path = path + '/valid/'

In [28]:
vgg = Vgg16()

/Users/yangzhang/anaconda/lib/python2.7/site-packages/keras/layers/core.py:621: UserWarning: `output_shape` argument not specified for layer lambda_1 and cannot be automatically inferred with the Theano backend. Defaulting to output shape `(None, 3, 224, 224)` (same as input shape). If the expected output shape is different, specify it via the `output_shape` argument.
  .format(self.name, input_shape))


In [32]:
batch_size = 4

In [33]:
no_of_epochs = 3

In [36]:
batches = vgg.get_batches(train_path, batch_size=batch_size)

Found 500 images belonging to 2 classes.


In [43]:
val_batches = vgg.get_batches(valid_path, batch_size=batch_size*2)

Found 20 images belonging to 2 classes.


In [44]:
vgg.finetune(batches)

In [45]:
vgg.model.optimizer.lr = 0.01

In [46]:
latest_weights_filename = None

In [48]:
for epoch in range(no_of_epochs):
    print 'Running epoch %d' % epoch
    vgg.fit(batches, val_batches, nb_epoch=1)
    latest_weights_filename='ft%d.h5' % epoch
    vgg.model.save_weights(results_path + latest_weights_filename)
print 'Completed %d fit operations' % no_of_epochs

Running epoch 0
Epoch 1/1
500/500 [==============================] - 277s - loss: 1.0753 - acc: 0.9040 - val_loss: 1.6300 - val_acc: 0.9000
Running epoch 1
Epoch 1/1
500/500 [==============================] - 304s - loss: 0.6942 - acc: 0.9460 - val_loss: 0.8059 - val_acc: 0.9500
Running epoch 2
Epoch 1/1
500/500 [==============================] - 290s - loss: 0.6482 - acc: 0.9520 - val_loss: 7.1173e-06 - val_acc: 1.0000
Completed 3 fit operations


# Make Prediction

In [51]:
batches, preds = vgg.test(test_path, batch_size=batch_size*2)

Found 12500 images belonging to 1 classes.


In [53]:
filenames = batches.filenames

In [54]:
save_array(results_path+'test_pred.dat', preds)
save_array(results_path+'filenames.dat', filenames)

# Submit Predictions to Kaggle!

In [29]:
preds = load_array(results_path+'test_pred.dat')
filenames = load_array(results_path+'filenames.dat')

In [30]:
is_dog = preds[:,1]

is_dog = is_dog.clip(min=0.05, max=0.95)

In [31]:
ids = np.array([int(flnm[flnm.find('/')+1:flnm.find('.')]) for flnm in filenames])

subm = np.stack([ids, is_dog], axis=1)

In [37]:
submission_file_name = results_path+'submission_1.csv'
np.savetxt(submission_file_name, subm, fmt='%d,%.5f', header='id,label', delimiter=',', comments='')

In [38]:
%%bash -s "$submission_file_name" "$KAGGLE_USER" "$KAGGLE_PW" 
kg submit $1 -u $2 -p $3 -m "Plain run of vgg on sample training data"